
El GCPD recolecta la información de casos policiales que acontecen en Ciudad Gótica.
Esta informacióñ se encuentra guarada en un archivo con el siguiente formato:

(fecha, id_caso, descripción, estado_caso, categoria, latitud, longitud)

Los posibels estados que puede tener un caso son 1: caso abierto, 2: caso resuelto,
3: cerrado sin resolucion.

Las fechas se encuentran en el formato YYYY-MM-DD.

Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuales
casos fue activada la batiseñal para pedir ayuda del vigilante, Batman.
Esta información se encuentra en un archivo con el siguiente formato (id_caso, respeusta),
siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.

El sector encargado de las estadísticas oficiales del GCPD quiere analizar las siguientes situaciones:

a) Las categorias que hayan incrementado su tasa de resolución al menos un 10% en
el último trimestre, con respecto al trimestre anterior.

b) Tasa de participación de Batman por categoria, para los delitos contra la
propiedad (que enmarcan las categorías incendio intencional, robo, hurto y robo de vehiculos).

#como mi df tiene categorias numericas (xd) hago de cuenta que las categorias pedidas son la 1, 2, 3 y 4

Resolver ambas consultas utilizando Pandas.

LA SOLUCION DE DEL MAZO USA UNA PIVOT TABLE DE COLUMNA TRIMESTRE, INDICE CATEGORIA Y
VALORES Y DE VALUES LA FUNCION QUE CALCULA CANT DE CASOS RESUELTOS EN PORCENTAJE

In [60]:
import pandas as pd

gcpdDf = pd.read_csv('/home/marcos/PycharmProjects/PracticaPandas/Ej6a.csv')
gordonDf = pd.read_csv('/home/marcos/PycharmProjects/PracticaPandas/Ej6b.csv')

gcpdDf['fecha'] = pd.to_datetime(gcpdDf['fecha'], errors='coerce', format='%Y-%m-%d')
gcpdDf['trimestre'] = gcpdDf['fecha'].dt.month % 4
gcpdAskedTrimesters = gcpdDf[gcpdDf['trimestre'].isin([2,3]) & (gcpdDf['fecha'].dt.year == 2018)] #habria que comparar con el anio actual enrealidad, pero mi df no tiene datos xd
gcpdPrevTrimData = gcpdAskedTrimesters.groupby(by=['categoria', 'trimestre'])\
                    .agg({'estado_caso':(lambda x: (x == 2).mean() * 100)})
gcpdPrevTrimData = gcpdPrevTrimData.unstack().droplevel(axis=1, level=0)
gcpdPrevTrimData['Incremented Rate'] = (gcpdPrevTrimData[3] - gcpdPrevTrimData[2]) > 10
gcpdPrevTrimData

trimestre,2,3,Incremented Rate
categoria,,,
1,0.0,0.000000,False
2,100.0,0.000000,False
3,0.0,NaN,False
4,0.0,66.666667,True
5,0.0,NaN,False
6,0.0,NaN,False
7,50.0,NaN,False
8,NaN,33.333333,False
9,100.0,100.000000,False


In [61]:
gcpdReqCategories = gcpdDf[gcpdDf['categoria'].isin([1,4])]
gcpdReqCategories = gcpdReqCategories.merge(gordonDf, how='inner', on='id_caso').groupby(by='categoria') \
                        .agg({'respuesta':(lambda x: (x == 1).mean() * 100)})
gcpdReqCategories.rename(columns={'respuesta':'tasa de respuesta'}, inplace=True)
gcpdReqCategories

,tasa de respuesta
categoria,
1,51.020408
4,54.954955
